In [1]:
# Ignore warning
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Set device to gpu if available
import torch
device = 'cpu'
if torch.cuda.is_available:
  print('All good, a Gpu is available')
  device = torch.device("cuda:0")  
else:
  print('Please set GPU via Edit -> Notebook Settings.')
  

All good, a Gpu is available


In [3]:
!nvidia-smi

Sat May 14 07:18:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install -q torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-geometric
!pip install -q ogb

     |████████████████████████████████| 7.9 MB 8.8 MB/s 
     |████████████████████████████████| 3.5 MB 8.3 MB/s 
     |████████████████████████████████| 407 kB 9.9 MB/s 
     |████████████████████████████████| 78 kB 4.8 MB/s 


In [5]:
# Import modules
import networkx as nx
from torch_geometric.utils.convert import to_networkx

import tqdm
import time
import sys
import os
import math
import random

import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch_geometric.utils import negative_sampling

import numpy as np

import pandas as  pd

import seaborn as sns

import matplotlib.pyplot as plt
from matplotlib import cm, colors

from ogb.linkproppred import PygLinkPropPredDataset, Evaluator

import torch_geometric.transforms as T
from torch_sparse import SparseTensor
from torch_geometric.nn import GCNConv, SAGEConv

from sklearn.metrics import roc_auc_score

## Reproducibility
When **developing and debugging** Neural Networks is desirable to have a deterministic behaviour. For this reason, we are going to disable all the sources of randomness. Please note that completely reproducible results are not guaranteed across PyTorch releases, individual commits or different platforms. You can find more detailed information at this [page](https://pytorch.org/docs/stable/notes/randomness.html).
Please note that the flag `cudnn.benchmark = False` disable the auto-tuner that selects the optimal set of algorithms for your hardware and usually leads to slower runtime. 

In [6]:
def fix_random(seed: int) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True        

In [7]:
fix_random(42)

## The Dataset

In [8]:
# dataset = PygLinkPropPredDataset(name="ogbl-ddi", root='datasets/')
# evaluator = Evaluator(name='ogbl-ddi')

In [9]:
# dataset = PygLinkPropPredDataset(name="ogbl-collab", root='datasets/')
# evaluator = Evaluator(name='ogbl-collab')

In [10]:
dataset = PygLinkPropPredDataset(name="ogbl-ppa", root='datasets/')
evaluator = Evaluator(name='ogbl-ppa')

Downloaded 0.38 GB: 100%|██████████| 388/388 [00:04<00:00, 87.30it/s]


Extracting datasets/ppassoc.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00,  1.81it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 5637.51it/s]


Saving...


Done!


In [11]:
data = dataset[0]
edge_index = data.edge_index.to(device)
data.x = data.x.to(torch.float)
if data.edge_weight is not None:
    data.edge_weight = data.edge_weight.view(-1).to(torch.float)

transform = T.Compose([
    # T.NormalizeFeatures(),
    T.RemoveIsolatedNodes(),
    T.ToSparseTensor(),
])
data = transform(data)

In [12]:
print()
print(f'Dataset: {dataset.name}:')
print('==================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

print()
print(data)
print('=======================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: ogbl-ppa:
Number of graphs: 1
Number of features: 58
Number of classes: 0

Data(num_nodes=576289, x=[576289, 58], adj_t=[576289, 576289, nnz=42463862])
Number of nodes: 576289
Number of edges: 42463862
Average node degree: 73.69
Has isolated nodes: False
Has self-loops: False
Is undirected: True


##Model Definition

In [13]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, dropout):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels, cached=True))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, cached=True))
        self.convs.append(GCNConv(hidden_channels, out_channels, cached=True))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x

In [14]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x

In [15]:
from torch.nn import Sequential, Linear, Dropout
class SAGE2(torch.nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim, n_layers, p_drop, skip_conn=False, post_proc_MLP=False):
        super(SAGE2, self).__init__()

        if n_layers < 1:
            raise AssertionError('Number or convolution layer must be at least 1')

        self.p_drop = p_drop
        self.n_layers = n_layers
        self.skip_conn = skip_conn

        self.convs = torch.nn.ModuleList()
        for i in range(n_layers):
            conv = SAGEConv(in_dim, hid_dim, normalize=True, aggr="add")
            in_dim = hid_dim
            self.convs.append(conv)

        self.post_proc_MLP = post_proc_MLP
        if post_proc_MLP:
            self.post_MLP = Sequential(
                Linear(in_dim, hid_dim), Dropout(self.p_drop),
                Linear(hid_dim, out_dim)
            )
        else:
            self.convs[-1] = SAGEConv(in_dim, out_dim, normalize=True, aggr="add")

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        if self.post_proc_MLP:
            self.post_MLP.reset_parameters()

    def forward(self, x, adj_t):
        x_prev = None
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)

            if self.skip_conn and x_prev is not None:
                x = x + x_prev
            
            x = F.relu(x)
            x = F.dropout(x, p=self.p_drop, training=self.training)
            x_prev = x
            
        if self.post_proc_MLP:
            x = self.convs[-1](x, adj_t)
            x = self.post_MLP(x)
        else:
            x = self.convs[-1](x, adj_t)

        return x

The LinkPredictor takes the embedding vector of 2 nodes ($h_i$, $h_j \in \mathbb{R}^d$ ) and computes the probability score of whether there exists a link between the 2 nodes.

$$\text{Probability of edge between node i and node j = MLP}(h_i\odot h_j)$$

Note: $x \odot y$ is the element-wise product (Hadamard product) of vectors $x$ and $y$

In [16]:
class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,  dropout):
        super(LinkPredictor, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()

    def forward(self, x_i, x_j):
        x = x_i * x_j
        for lin in self.lins[:-1]:
            x = lin(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.sigmoid(x)

## Train and evaluation

The model is trained by jointly maximizing the probabilty prediction of correct edges and minimizing the probability prediciton of the incorrect edges:

$Loss = -\log(prob\_pos\_edges  + ϵ) - \log(1-prob\_neg\_edges + ϵ)$ where the $ϵ=10^{-15}$ is added to each term for numerical stability

In [17]:
def train(model, predictor, node_feats, adj_t, split_edge, optimizer, batch_size):
    model.train()
    predictor.train()

    pos_train_edge = split_edge['train']['edge'].to(device)

    total_loss = total_examples = 0
    for edge_id in DataLoader(range(pos_train_edge.size(0)), batch_size, shuffle=True):
        optimizer.zero_grad()

        node_emb = model(node_feats, adj_t)
        

        # Predict the class probabilities on the batch of positive edges using link_predictor
        pos_edge = pos_train_edge[edge_id].t()
        pos_pred = predictor(node_emb[pos_edge[0]], node_emb[pos_edge[1]])

        # Trivial random sampling.
        #neg_edge = torch.randint(0, node_feats.size(0), (batch_size,), device=device)
        # Sample negative edges (same number as number of positive edges) and predict class probabilities 
        neg_edge = negative_sampling(edge_index, num_nodes=node_feats.size(0), num_neg_samples=batch_size, method='sparse')
        neg_pred = predictor(node_emb[neg_edge[0]], node_emb[neg_edge[1]])

         # Compute the corresponding negative log likelihood loss on the positive and negative edges
        loss = -torch.log(pos_pred + 1e-15).mean() - torch.log(1 - neg_pred + 1e-15).mean()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)

        optimizer.step()

        num_examples = pos_pred.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

    return total_loss / total_examples

The performance of a link predictor is evaluated through the metric Hits@K.

Hits@K = Fraction of correct links in the top K links (with respect to their scores)

In [18]:
@torch.no_grad()
def test(model, predictor, data, adj_t, split_edge, evaluator, batch_size):
    model.eval()
    predictor.eval()

    h = model(data, adj_t)

    pos_train_edge = split_edge['train']['edge'].to(h.device)
    pos_valid_edge = split_edge['valid']['edge'].to(h.device)
    neg_valid_edge = split_edge['valid']['edge_neg'].to(h.device)
    pos_test_edge = split_edge['test']['edge'].to(h.device)
    neg_test_edge = split_edge['test']['edge_neg'].to(h.device)

    # ------------------ TRAIN ------------------- #
    # Positive edges prediction
    pos_train_preds = []
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size):
        edge = pos_train_edge[perm].t()
        pos_train_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_train_pred = torch.cat(pos_train_preds, dim=0)

    # ---------------- VALIDATION ---------------- #
    # Positive edges prediction
    pos_valid_preds = []
    for perm in DataLoader(range(pos_valid_edge.size(0)), batch_size):
        edge = pos_valid_edge[perm].t()
        pos_valid_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_valid_pred = torch.cat(pos_valid_preds, dim=0)
    # Negative sampled edges prediction
    neg_valid_preds = []
    for perm in DataLoader(range(neg_valid_edge.size(0)), batch_size):
        edge = neg_valid_edge[perm].t()
        neg_valid_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    neg_valid_pred = torch.cat(neg_valid_preds, dim=0)

    # ------------------- TEST ------------------- #
    # Positive edges prediction
    pos_test_preds = []
    for perm in DataLoader(range(pos_test_edge.size(0)), batch_size):
        edge = pos_test_edge[perm].t()
        pos_test_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_test_pred = torch.cat(pos_test_preds, dim=0)
    # Negative sampled edges prediction
    neg_test_preds = []
    for perm in DataLoader(range(neg_test_edge.size(0)), batch_size):
        edge = neg_test_edge[perm].t()
        neg_test_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    neg_test_pred = torch.cat(neg_test_preds, dim=0)

    results = {}
    # hits@K
    for K in [20, 50, 100]:
        evaluator.K = K
        train_hits = evaluator.eval({
            'y_pred_pos': pos_train_pred,
            'y_pred_neg': neg_valid_pred,
        })[f'hits@{K}']
        valid_hits = evaluator.eval({
            'y_pred_pos': pos_valid_pred,
            'y_pred_neg': neg_valid_pred,
        })[f'hits@{K}']
        test_hits = evaluator.eval({
            'y_pred_pos': pos_test_pred,
            'y_pred_neg': neg_test_pred,
        })[f'hits@{K}']

        results[f'hits@{K}'] = (round(train_hits,2), round(valid_hits,2), round(test_hits,2))

    # auc
    # results['valid_auc'] = roc_auc_score(
    #     y_true = np.concatenate((np.ones(pos_valid_pred.size(0), dtype=int), np.zeros(neg_valid_pred.size(0), dtype=int))),
    #     y_score = np.concatenate((pos_valid_pred.numpy(), neg_valid_pred.numpy())) 
    # )
    # results['test_auc'] = roc_auc_score(
    #     y_true = np.concatenate((np.ones(pos_test_pred.size(0), dtype=int), np.zeros(neg_test_pred.size(0), dtype=int))),
    #     y_score = np.concatenate((pos_test_pred.numpy(), neg_test_pred.numpy()))
    # )

    return results  

In [ ]:
USE_SAGE = True
USE_VAL_EDGES_AS_INPUT = False

BATCH_SIZE = 128 * 1024
INPUT_DIM = dataset.num_features
OUTPUT_DIM = dataset.num_classes
EPOCHS = 25

LR = 5e-3
WD = 1e-6
EVAL_STEPS = 5


split_edge = dataset.get_edge_split()

# Use training + validation edges for inference on test set.
if USE_VAL_EDGES_AS_INPUT:
    val_edge_index = split_edge['valid']['edge'].t()
    full_edge_index = torch.cat([edge_index, val_edge_index], dim=-1)
    data.full_adj_t = SparseTensor.from_edge_index(full_edge_index).t()
    data.adj_t = data.full_adj_t.to_symmetric()

data = data.to(device)

best_test_hits = 0
best_model_history = None
best_model_weight = None

from sklearn.model_selection import ParameterGrid
configs = {
    'USE_EMBEDDING_LAYER': [True, False],
    'HIDDEN_DIM' : [32],
    'DROPOUT' : [0],
    'N_LAYERS' : [3],
    'SKIP_CONN' : [True],
    'POST_PROC_MLP' : [True],
}
param_grid = ParameterGrid(configs)
print(f'Number of combinations: {len(param_grid)}')
print('------------------')
for dict_ in param_grid:
    print(f'Trying configuration {dict_}')
    parameter_list = []

    #INPUT DATA
    if dict_['USE_EMBEDDING_LAYER']:
        if data.x is not None:
            emb = torch.nn.Embedding(data.num_nodes, INPUT_DIM).to(device)
            emb.weight.data.copy_(data.x) # copy data features as initial embedding
        else:
            emb = torch.nn.Embedding(data.num_nodes, dict_['HIDDEN_DIM']).to(device)  
        input_data = emb.weight
        parameter_list += (list(emb.parameters()))
    else:
        input_data = data.x

    #MODEL
    if USE_SAGE:
        model = SAGE2(INPUT_DIM, dict_['HIDDEN_DIM'], dict_['HIDDEN_DIM'], dict_['N_LAYERS'], dict_['DROPOUT'], skip_conn=True).to(device)
    else:
        model = GCN(INPUT_DIM, dict_['HIDDEN_DIM'], dict_['HIDDEN_DIM'], dict_['N_LAYERS'], dict_['DROPOUT']).to(device)
    parameter_list += (list(model.parameters()))

    #LINK PREDICTOR
    link_predictor = LinkPredictor(dict_['HIDDEN_DIM'], dict_['HIDDEN_DIM'], 1,  dict_['N_LAYERS'], dict_['DROPOUT']).to(device)
    parameter_list += (list(link_predictor.parameters()))

    optimizer = torch.optim.Adam(parameter_list, lr=LR, weight_decay=WD)

    #TRAIN & EVALUATE
    history = {}
    history['loss'] = []
    history['train_hits@20'] = []; history['train_hits@50'] = []; history['train_hits@100'] = []
    history['valid_hits@20'] = []; history['valid_hits@50'] = []; history['valid_hits@100'] = []
    history['test_hits@20']  = []; history['test_hits@50']  = []; history['test_hits@100']  = []
    pbar = tqdm.tqdm(range(EPOCHS))
    for epoch in pbar:
        loss = train(model, link_predictor, input_data, data.adj_t, split_edge, optimizer, BATCH_SIZE) 
        history['loss'].append(loss)

        if (epoch + 1) % EVAL_STEPS == 0 or epoch == 0:
            results = test(model, link_predictor, input_data, data.adj_t, split_edge, evaluator, BATCH_SIZE)
            for key, result in results.items():
                train_hits, valid_hits, test_hits = result
                history[f'train_{key}'].append(train_hits)
                history[f'valid_{key}'].append(valid_hits)
                history[f'test_{key}'].append(test_hits)

        description = f"Epoch {(epoch + 1):02d} - loss: {loss:.4f} - result: {results}\t"
        pbar.set_description(description, refresh=True)

    print('------------------')
    
    if max(history['test_hits@100']) > best_test_hits:
        best_parameters = dict_
        best_model_history = history
        best_model_weights = model.state_dict()
        best_test_hits = max(history['test_hits@100'])
    
    del model; del link_predictor
    torch.cuda.empty_cache()

Number of combinations: 2
------------------
Trying configuration {'DROPOUT': 0, 'HIDDEN_DIM': 32, 'N_LAYERS': 3, 'POST_PROC_MLP': True, 'SKIP_CONN': True, 'USE_EMBEDDING_LAYER': True}


  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
print(f'Best parameters: {best_parameters}')

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(np.arange(1, EPOCHS+1), best_model_history['loss'], label="train loss")
plt.plot(np.arange(0, EPOCHS+1, 5), best_model_history['train_hits@100'], label="train Hits@100")
plt.plot(np.arange(0, EPOCHS+1, 5), best_model_history['valid_hits@100'], label="val Hits@100")
plt.plot(np.arange(0, EPOCHS+1, 5), best_model_history['test_hits@100'], label="test Hits@100")
#plt.plot(np.arange(0, EPOCHS+1, 10), best_model_history['test_auc'], label="test auc")
plt.title('Link Prediction on OGB-ddi using GraphSAGE GNN')
plt.xlabel('Epochs')
plt.ylim(0., 1.)
plt.legend(loc='upper left')
plt.show()